In [48]:
#imports libraries that will be utilized
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import requests
import scipy.stats as sts
import scipy as spy
import scipy.integrate as integrate
import scipy.optimize as opt
import random
import numpy.linalg as lin
import pickle as pkl
import os
import dateparser

In [2]:
#Loads Generators Stack
#FuelTypes = HYDRO, GAS, WIND, NUCLEAR
#HeatRate [BTU/kWh]
#EmmisionRate [lb/MWh]
GenStack0=pd.read_csv('GenStack.csv',sep=",")
GenStack0.columns=["Gen","FuelType","PMax","HeatRate","EmissionRate","Cost"]

GenStackCO2=pd.read_csv('GenStackCO2v0.csv',sep=",")
GenStackCO2.columns=["Gen","FuelType","PMax","HeatRate","EmissionRate","Cost"]

SCC=36

with open('GenStack0','wb') as f:
    pkl.dump(GenStack0, f)

with open('GenStackCO2','wb') as f:
    pkl.dump(GenStackCO2, f)

In [201]:
LoadProfile0=pd.DataFrame(np.zeros((0,5)))
LoadProfile0.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
TempProfile=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))                
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)

In [202]:
LoadProfile=[]
count=0
hrLoad=0
for pos in range (0,len(TempProfile)):
    if pos == len(TempProfile)-1:
        hrLoad=hrLoad+TempProfile[pos]
        LoadProfile.append(hrLoad)
    if count==11:
        LoadProfile.append(hrLoad)
        count=0
        hrLoad=0
    hrLoad=hrLoad+TempProfile[pos]
    count=count+1
    
with open('LoadProfile','wb') as f:
    pkl.dump(LoadProfile, f)

In [5]:
MonthNaturalGasPrices=[]
with open(os.path.join("Data","MonthlyNG_prices_17_18.csv")) as f:
    NaturalGasPricesTemp=pd.read_csv(f,sep=",",header=None)
    
MonthNaturalGasPrices=NaturalGasPricesTemp.iloc[:,0]

In [6]:
HrNaturalGasPrices=[]

yearcount=0
for m in range (1,25):
    if m>12:
        m=m-12
        yearcount=1
    if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
        for d in range (1,(31*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])
    if m==4 or m==6 or m==9 or m==11:
        for d in range (1,(30*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])
    if m==2:
        for d in range (1,(28*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])

with open('HrNaturalGasPrices','wb') as f:
    pkl.dump(HrNaturalGasPrices, f)

In [7]:
PSeries0=pd.DataFrame(np.zeros((0,6)))
PSeries0.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
TempPSeries=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)

In [8]:
PSeries=[]
count=0
TotCosthr=0
posLoad=0
for pos in range (0,len(TempPSeries)):
    if pos == len(TempProfile)-1:
        TotCosthr=TotCosthr+(TempProfile[pos]*TempPSeries[posLoad])
        PSeries.append(TotCosthr)
    if count==15:
        PSeries.append(TotCosthr)
        count=0
        TotCosthr=0
    if count==0 or count==1 or count==2 or count==3 or count==5 or count==6 or count==7 or count==8 or count==9 or count==10 or count==14:
        TotCosthr=TotCosthr+(TempProfile[posLoad]*TempPSeries[pos])
        posLoad=posLoad+1
    count=count+1

for row in range (0,len(PSeries)):
    PSeries[row]=PSeries[row]/LoadProfile[row]
    if PSeries[row]<0:
        PSeries[row]=0
        
with open('PSeries','wb') as f:
    pkl.dump(PSeries, f)

In [86]:
Exchange0=pd.DataFrame(np.zeros((0,6)))
Exchange0.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
TempExchange=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)

In [198]:
Exchange = pd.DataFrame(Exchange0)
Exchange['TimeStamp']=pd.to_datetime(Exchange['TimeStamp'])
Exchange['PTID']=np.array(Exchange['PTID'])
Exchange.set_index(Exchange['PTID'],inplace=True)
Exchange=Exchange.drop([23330,23326,23319,23320,23314,23315,23312])
Exchange.set_index(Exchange['TimeStamp'],inplace=True)
Exchange=Exchange.groupby([Exchange.index.year, Exchange.index.month, Exchange.index.day, Exchange.index.hour]).sum()
Exchange['Flow']=Exchange['Flow']/12
Exchange=np.array(Exchange['Flow'])
Daylight2017=Exchange[(31*24)+(28*24)+(11*24)+2]
Daylight2018=Exchange[(365*24)+(31*24)+(28*24)+(10*24)+2]

TempHrExchange=[]
for row in range (0,len(Exchange)):
    if row==(31*24)+(28*24)+(11*24)+2:
        TempHrExchange.append(Daylight2017)
    if row==(365*24)+(31*24)+(28*24)+(10*24)+2:
        TempHrExchange.append(Daylight2018)
    TempHrExchange.append(Exchange[row])
    
Exchange=TempHrExchange

In [205]:
ModLoadProfile=np.array(LoadProfile[:])-np.array(Exchange[:])


In [9]:
def MarginalGen(load):
    Maxgen=0
    genid=0
    while Maxgen<load and genid<len(GenStack0['PMax']):
        Maxgen=Maxgen+GenStack0['PMax'][genid]
        genid=genid+1
    return genid-1
def MarginalGenCO2(load):
    Maxgen=0
    genid=0
    while Maxgen<load and genid<len(GenStackCO2['PMax']):
        Maxgen=Maxgen+GenStackCO2['PMax'][genid]
        genid=genid+1
    return genid-1

In [10]:
def MarginalCost(marGen,NGPrice,csource):
    marCos=((GenStack0['HeatRate'][marGen])/1000000)*1000*(NGPrice+csource)
    return marCos
def MarginalCostCO2(marGen,NGPrice, csource):
    marCos=(((GenStackCO2['HeatRate'][marGen])/1000000)*1000*(NGPrice+csource))+(GenStackCO2['Cost'][marGen])
    return marCos

In [11]:
#Initial weighting matrix
W=np.eye(4)
sim=1200

sigma_0=5
mu_0=5
rho_0=8/9

#Generates matrix with random cumulative probability values
np.random.seed(25)  # Set the random number seed so it gives same answers every time
S_0 = sts.uniform.rvs(0, 1, size=(len(PSeries),sim))

#Function that calculates error based on a cumulative probability value
def epsilonma(matrix,sigma):
    epsm=sts.norm.ppf(matrix,loc=0,scale=sigma)
    return epsm

#Function that calculates a simulated CSource's based on errors, rho, mu and an initial CSource
def simC(errors,rho,mu,cSource0):
    simulatedC=np.array(errors)
    simulatedC[0,:]=np.array(rho*cSource0+(1-rho)*mu+errors[0,:])
    
    for l in range (1,len(PSeries)):
        simulatedC[l,:]=np.array(rho*simulatedC[l-1,:]+(1-rho)*mu+errors[l,:])
    
    return simulatedC

#Function that calculates a simulated P(t) based on Load and CSource
def simP(simC,Load):
    simulatedP=np.array(simC)
    LoadProf=np.array(Load)
    
    for row in range (0, len(PSeries)):
        MarGen=MarginalGen(LoadProf[row])
        simulatedP[row,:]=MarginalCost(MarGen,HrNaturalGasPrices[row],simC[row,:])
    
    return simulatedP

#Calculates data moments
m1=np.mean(PSeries)
m2=np.var(np.array(PSeries[:])*np.array(LoadProfile[:]))
corrPL, pvalPL = sts.pearsonr(PSeries,LoadProfile)
m3=corrPL
modPt=np.zeros((len(PSeries)-1))
modPt1=np.zeros((len(PSeries)-1))

for count in range (0, len(PSeries)-1):
    modPt[count]=PSeries[count]
    modPt1[count]=PSeries[count+1]

corrPt, pvalPt=sts.pearsonr(modPt,modPt1)
m4=corrPt

#Defines optimization criterion
def crit(params):
    rho,mu,sigma = params

    #Creates objects to store simulated moments
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    modPst=np.zeros((len(PSeries)-1,sim))
    modPst1=np.zeros((len(PSeries)-1,sim))
    modPt=np.zeros((len(PSeries)-1))
    modPt1=np.zeros((len(PSeries)-1))
    
    
    cSource0=mu
    
    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    Cst=np.array(simC(errors,rho,mu,cSource0))
    Pst=np.array(simP(Cst,ModLoadProfile))

    
    #Calculates simulated moments
    for i in range (0, sim):
    
        sm1.append(np.mean(Pst[:,i]))
        sm2.append(np.var(Pst[:,i]*np.array(LoadProfile[:])))

        corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(LoadProfile[:]))
        sm3.append(corrm3)
        
        for count in range (0, len(PSeries)-1):
            modPst[count,i]=Pst[count,i]
            modPst1[count,i]=Pst[count+1,i]
        
        corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
        sm4.append(corrm4)
        

        
    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4

    #Creates array of errors
    err=np.array([err1, err2, err3, err4])
    
    #Calculates estimation criterion
    estimator=err.T @ W @ err
    
    return estimator

#Performs optimization starting with initial parameters
params_0=np.array([rho_0,mu_0, sigma_0])
results_uncstr = opt.minimize(crit, params_0, method='L-BFGS-B',bounds=((-0.99, 0.99),(0.01,50),(0.01,10)),tol=1e-5)
rho, mu, sigma = results_uncstr.x

#Prints estimated parameters
print("Estimated Rho = ", rho)
print("Estimated Mu = ", mu)
print("Estimated Sigma = ", sigma)

Estimated Rho =  -0.9898378743005439
Estimated Mu =  0.5894008879534631
Estimated Sigma =  0.09670474361885388


In [12]:
#Calculates final values of moments with optimized parameters (repeats the process described in
#   "optimization criteria")
sm1=[]
sm2=[]
sm3=[]
sm4=[]
modPst=np.zeros((len(PSeries)-1,sim))
modPst1=np.zeros((len(PSeries)-1,sim))
modPt=np.zeros((len(PSeries)-1))
modPt1=np.zeros((len(PSeries)-1))


cSource0=mu

#Calculates variables as per problem set's equations
errors=np.array(epsilonma(S_0, sigma))
Cst=np.array(simC(errors,rho,mu,cSource0))
Pst=np.array(simP(Cst,LoadProfile))


#Calculates simulated moments
for i in range (0, sim):

    sm1.append(np.mean(Pst[:,i]))
    sm2.append(np.var(Pst[:,i]*np.array(LoadProfile[:])))

    corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(LoadProfile[:]))
    sm3.append(corrm3)

    for count in range (0, len(PSeries)-1):
        modPst[count,i]=Pst[count,i]
        modPst1[count,i]=Pst[count+1,i]

    corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
    sm4.append(corrm4)



#Calculates errors (Difference between simulated moments and data moments)
err1=(np.mean(sm1)-m1)/m1
err2=(np.mean(sm2)-m2)/m2
err3=(np.mean(sm3)-m3)/m3
err4=(np.mean(sm4)-m4)/m4

#Creates array of errors
err=np.array([err1, err2, err3, err4])

#Prints final vector of moment differences
print("Vector of differences = ", err)

#Prints value of final estimator
print("Criterion function value =",crit(results_uncstr.x))

#Calculates errors matrix
#1. Defines function for calculating errors
def s_errors(params):
    
    rho, mu, sigma = params
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    modPst=np.zeros((len(PSeries)-1,sim))
    modPst1=np.zeros((len(PSeries)-1,sim))
    modPt=np.zeros((len(PSeries)-1))
    modPt1=np.zeros((len(PSeries)-1))


    cSource0=mu

    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    Cst=np.array(simC(errors,rho,mu,cSource0))
    Pst=np.array(simP(Cst,LoadProfile))


    #Calculates simulated moments
    for i in range (0, sim):

        sm1.append(np.mean(Pst[:,i]))
        sm2.append(np.var(Pst[:,i]*np.array(LoadProfile[:])))

        corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(LoadProfile[:]))
        sm3.append(corrm3)

        for count in range (0, len(PSeries)-1):
            modPst[count,i]=Pst[count,i]
            modPst1[count,i]=Pst[count+1,i]

        corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
        sm4.append(corrm4)



    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4

    #Creates array of errors
    err=np.array([err1, err2, err3, err4])
    
    return err

#2. Generates Jacobian Matrix
npar=3
nmom=4
h=1e-14
Ematrix=np.zeros((nmom, npar))


for col in range (0,npar):
    derivparamup=np.array([rho, mu, sigma])
    derivparamup[col]=derivparamup[col]+h
    derivparamdown=np.array([rho, mu, sigma])
    derivparamdown[col]=derivparamdown[col]-h
    deltaerror=s_errors(derivparamup)-s_errors(derivparamdown)
    for row in range (0,nmom):
        Ematrix[row,col]=deltaerror[row]/(2*h)
        
        
        
cov2=(1/sim)*lin.pinv(Ematrix.T@ W @ Ematrix)

StandardErra=[]

#3. Calculates and prints Standard errors. Order: m1, m2, m3, m4
for row in range (0,npar):
    StandardErra.append((cov2[row,row])**1/2)

print("Standard Errors of Parameter Vector = ", StandardErra)

Vector of differences =  [ 0.22221612 -0.77906194 -0.14793936 -0.02434229]
Criterion function value = 0.6787961081870527
Standard Errors of Parameter Vector =  [0.26602208940242567, 0.004284197370417988, 5.976916573868696]


In [13]:
results_uncstr

      fun: 0.6787961081870527
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00211735,  0.00011905, -0.00152933])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 84
      nit: 11
   status: 0
  success: True
        x: array([-0.98983787,  0.58940089,  0.09670474])

In [14]:
prodHydro=np.zeros((len(PSeries)))
revHydro=np.zeros((len(PSeries)))
prodHydroCO2=np.zeros((len(PSeries)))
revHydroCO2=np.zeros((len(PSeries)))
prodNuclear=np.zeros((len(PSeries)))
revNuclear=np.zeros((len(PSeries)))
prodNuclearCO2=np.zeros((len(PSeries)))
revNuclearCO2=np.zeros((len(PSeries)))
prodWind=np.zeros((len(PSeries)))
revWind=np.zeros((len(PSeries)))
prodWindCO2=np.zeros((len(PSeries)))
revWindCO2=np.zeros((len(PSeries)))
prodGas=np.zeros((len(PSeries)))
revGas=np.zeros((len(PSeries)))
prodGasCO2=np.zeros((len(PSeries)))
revGasCO2=np.zeros((len(PSeries)))
emissions0=np.zeros((len(PSeries)))
emissionsCO2=np.zeros((len(PSeries)))
consumersPay=np.zeros((len(PSeries)))
consumersPayCO2=np.zeros((len(PSeries)))
taxCollectedCO2=np.zeros((len(PSeries)))
HrLoadCO2=np.zeros((len(PSeries)))

DemandElasticity=-0.3

for row in range (0,len(PSeries)):
    
    CSourcet=np.mean(Cst[row,:])
    HrNGPrice=HrNaturalGasPrices[row]
    
    Marginal0=MarginalGen(LoadProfile[row])
    generation0=0
    for gen in range (0,Marginal0):
        generation0=generation0+GenStack0['PMax'][gen]

    Pt=MarginalCost(Marginal0,HrNGPrice,CSourcet)
    consumersPay[row]=Pt*LoadProfile[row]
    
    errorLoop=1
    tolerance=1e-5
    TempCounter=0
    Konstant=LoadProfile[row]/(Pt**DemandElasticity)
    LoadCO2_0=LoadProfile[row]
    MarginalCO2_0=MarginalGenCO2(LoadCO2_0)
    PCO2t_0=MarginalCostCO2(MarginalCO2_0,HrNGPrice,CSourcet)
    
    while errorLoop>tolerance:
        LoadCO2=Konstant*(PCO2t_0**DemandElasticity)
        MarginalCO2_0=MarginalGenCO2(LoadCO2)
        PCO2t=MarginalCostCO2(MarginalCO2_0,HrNGPrice,CSourcet)

        errorLoop=np.abs(PCO2t-PCO2t_0)
        LoadCO2=Konstant*(PCO2t**DemandElasticity)
        PCO2t_0=PCO2t
        TempCounter=TempCounter+1
        if errorLoop<tolerance or TempCounter>10:
            break
    
    MarginalCO2=MarginalGenCO2(LoadCO2)
    generationCO2=0
    for gen in range (0,MarginalCO2):
        generationCO2=generationCO2+GenStackCO2['PMax'][gen]

    PCO2t=MarginalCostCO2(MarginalCO2,HrNGPrice,CSourcet)
    consumersPayCO2[row]=PCO2t*LoadCO2
    HrLoadCO2[row]=LoadCO2
    
    gen=0
    while gen<=Marginal0:
        if gen==Marginal0:
            Generation=LoadProfile[row]-generation0
        if gen<Marginal0:
            Generation=GenStack0['PMax'][gen]
            
        if GenStack0['FuelType'][gen]=="HYDRO":
            prodHydro[row]=prodHydro[row]+Generation
            revHydro[row]=revHydro[row]+(Generation*Pt)
        if GenStack0['FuelType'][gen]=="WIND":
            prodWind[row]=prodWind[row]+Generation
            revWind[row]=revWind[row]+(Generation*Pt)
        if GenStack0['FuelType'][gen]=="NUCLEAR":
            prodNuclear[row]=prodNuclear[row]+Generation
            revNuclear[row]=revNuclear[row]+(Generation*Pt)
        if GenStack0['FuelType'][gen]=="GAS":
            prodGas[row]=prodGas[row]+Generation
            revGas[row]=revGas[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
            emissions0[row]=emissions0[row]+(Generation*GenStack0['EmissionRate'][gen]*453.592/(1000000))
            
        gen=gen+1
        
        
    gen=0
    while gen<=MarginalCO2:
        if gen==MarginalCO2:
            Generation=LoadCO2-generationCO2
        if gen<MarginalCO2:
            Generation=GenStackCO2['PMax'][gen]
            
        if GenStackCO2['FuelType'][gen]=="HYDRO":
            prodHydroCO2[row]=prodHydroCO2[row]+Generation
            revHydroCO2[row]=revHydroCO2[row]+(Generation*PCO2t)
        if GenStackCO2['FuelType'][gen]=="WIND":
            prodWindCO2[row]=prodWindCO2[row]+Generation
            revWindCO2[row]=revWindCO2[row]+(Generation*PCO2t)
        if GenStackCO2['FuelType'][gen]=="NUCLEAR":
            prodNuclearCO2[row]=prodNuclearCO2[row]+Generation
            revNuclearCO2[row]=revNuclearCO2[row]+(Generation*PCO2t)
        if GenStackCO2['FuelType'][gen]=="GAS":
            prodGasCO2[row]=prodGasCO2[row]+Generation
            revGasCO2[row]=revGasCO2[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet)))
            emissionsCO2[row]=emissionsCO2[row]+(Generation*GenStackCO2['EmissionRate'][gen]*453.592/(1000000))
            taxCollectedCO2[row]=taxCollectedCO2[row]+(Generation*GenStackCO2['EmissionRate'][gen]*SCC*453.592/(1000000))
        gen=gen+1

hydroOut=np.sum(prodHydro)
hydroProf=np.sum(revHydro)
hydroOutCO2=np.sum(prodHydroCO2)
hydroProfCO2=np.sum(revHydroCO2)
WindOut=np.sum(prodWind)
WindProf=np.sum(revWind)
WindOutCO2=np.sum(prodWindCO2)
WindProfCO2=np.sum(revWindCO2)
NuclearOut=np.sum(prodNuclear)
NuclearProf=np.sum(revNuclear)
NuclearOutCO2=np.sum(prodNuclearCO2)
NuclearProfCO2=np.sum(revNuclearCO2)
GasOut=np.sum(prodGas)
GasProf=np.sum(revGas)
GasOutCO2=np.sum(prodGasCO2)
GasProfCO2=np.sum(revGasCO2)
emiss=np.sum(emissions0)
emissCO2=np.sum(emissionsCO2)
consumerExp=np.sum(consumersPay)
consumerExpCO2=np.sum(consumersPayCO2)
taxCollCO2=np.sum(taxCollectedCO2)

In [15]:
print(hydroOut)
print(hydroProf)
print(hydroOutCO2)
print(hydroProfCO2)
print(WindOut)
print(WindProf)
print(WindOutCO2)
print(WindProfCO2)
print(NuclearOut)
print(NuclearProf)
print(NuclearOutCO2)
print(NuclearProfCO2)
print(GasOut)
print(GasProf)
print(GasOutCO2)
print(GasProfCO2)
print(emiss)
print(emissCO2)
print(consumerExp)
print(consumerExpCO2)
print(taxCollCO2)

52498042.271474384
2158663855.58681
52498042.271474384
2927866257.8510447
7702168.456970399
316705003.43090004
7702168.456970399
429557335.1256502
82334154.49584
3385493166.990787
82334154.49584
4591854903.283703
174950105.5757153
1624503438.616464
146621928.64479357
1758716045.2124052
72603537.00250733
58273359.40619619
13344279348.218292
16577783695.405853
2097840938.6230626


In [16]:
TotCost=0
for line in range(0,len(PSeries)):
    TotCost=TotCost+(PSeries[line]*LoadProfile[line])

print(TotCost/np.sum(LoadProfile))
print(consumerExp/np.sum(LoadProfile))

35.928374351770074
42.0312820800125


In [17]:
print(np.mean(Cst))

0.5894020741968894


In [18]:
ExpLoadProfile=pd.DataFrame(LoadProfile)
ExpLoadProfile.columns=['MWh']
with open('LoadProfile.xlsx', "wb+") as f:
    
    ExpLoadProfile.to_excel(f, index=False)
    f.close
    
ExpPSeries=pd.DataFrame(PSeries)
ExpPSeries.columns=['MWh']
with open('PSeries.xlsx', "wb+") as f:
    ExpPSeries.to_excel(f, index=False)
    f.close

In [19]:
0.41*0.15+0.3*0.45+0.5*0.14

0.2665

In [54]:
print(np.sum(HrLoadCO2)/(1e6*2))
print(np.sum(LoadProfile)/(1e6*2))
print(np.average(HrNaturalGasPrices))

144.57814693453915
158.7422354
3.784095890410959


In [55]:
np.average(HrNaturalGasPrices)+np.mean(Cst)

4.373497964607848

In [14]:
with open('results_uncstr','rb') as f:
    results_uncstr = pkl.load(f)
    
rho, mu, sigma = results_uncstr.x

#Prints estimated parameters
print("Estimated Rho = ", rho)
print("Estimated Mu = ", mu)
print("Estimated Sigma = ", sigma)

Estimated Rho =  0.6072882791238214
Estimated Mu =  0.6192191823720016
Estimated Sigma =  2.6726168777591908
